In [ ]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from collections import Counter
import evalml
from evalml import AutoMLSearch
from evalml.utils import infer_feature_types
import torch
from torch.autograd import Variable
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix,accuracy_score, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import roc_auc_score, confusion_matrix,accuracy_score, roc_curve, auc, precision_recall_curve
import time

https://www.va.gov/MS/Professionals/diagnosis/Kurtzke_Expanded_Disability_Status_Scale.asp

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)

In [ ]:
folder = '../ai_training_set'
filelists = os.listdir(folder)

for file in filelists:
    if file not in ['Updated Database.xlsx', '.ipynb_checkpoints', 'AI_Notes']:
        print("df_" + file[:-4] + "=pd.read_csv('" + folder +"/" + file + "')")
        exec("df_" + file[:-4] + "=pd.read_csv('" + folder +"/" + file + "')")
        print("display(" + "df_" + file[:-4] + ".head(10))")
        exec("display(" + "df_" + file[:-4] + ".head(10))")

In [ ]:
for file in filelists:
    print("df_" + file[:-4])

In [ ]:
for i in df_AI_notes.columns:
    print(i)

In [ ]:
print('df_AI_demographics')
display(df_AI_demographics.head(3))
print('df_AI_patient_facts')
display(df_AI_patient_facts.head(3))
print('df_AI_labs')
display(df_AI_labs.head(3))
print('df_AI_vitals')
display(df_AI_vitals.head(3))
print('df_AI_diagnoses')
display(df_AI_diagnoses.head(3))
print('df_AI_notes')
display(df_AI_notes.head(3))
print('df_AI_medications')
display(df_AI_medications.head(3))
print('df_AI_encounters')
display(df_AI_encounters.head(3))
print('df_AI_procedures')
display(df_AI_procedures.head(3))


In [ ]:
df_AI_medications

In [ ]:
# df_AI_medications[df_AI_medications['M_MedDict']=='Topiramate 100 MG Oral Tablet']
# df_AI_medications[df_AI_medications['M_MedDict']=='Topiramate 50 MG Oral Tablet']
df_AI_medications['M_ndc'].nunique(),\
df_AI_medications['M_ItemID'].nunique(),\
df_AI_medications['M_ID'].nunique()

### Patient EDSS scores


In [ ]:
df_edss_score_EDSS = pd.read_excel('../ai_training_set/Updated Database.xlsx', sheet_name='(5) EDSS')
df_edss_score_MRI = pd.read_excel('../ai_training_set/Updated Database.xlsx', sheet_name='(6) MRI')

df_edss_score_MRI = df_edss_score_MRI.dropna(subset=['record_id', 'allscripts_mrn'])

## record_id --- allscripts_mrn
record_id_TO_allscripts_mrn = {record_id: allscripts_mrn for record_id, allscripts_mrn in zip(df_edss_score_MRI['record_id'].to_list(), df_edss_score_MRI['allscripts_mrn'].to_list())}

df_edss_score_EDSS = df_edss_score_EDSS.replace({"record_id": record_id_TO_allscripts_mrn}).rename(columns={'record_id': 'MRN'})

df_edss_score_EDSS = df_edss_score_EDSS.dropna().reset_index(drop=True)

result_EDSS = dict()  #{patient: {date: value}}
for i in range(df_edss_score_EDSS.shape[0]):
    mrn = df_edss_score_EDSS.loc[i, 'MRN']
    date = df_edss_score_EDSS.loc[i, 'dov']
    edss = df_edss_score_EDSS.loc[i, 'visit_edss']
    if mrn not in result_EDSS:
        result_EDSS[mrn] = dict()
    if date not in result_EDSS[mrn]:
        result_EDSS[mrn][date] = 0
    result_EDSS[mrn][date] = edss
    
result_EDSS



In [ ]:
a = Counter(df_AI_labs['IR_QO'])
# print(a.most_common(80))
labs_selected = [x[0] for x in a.most_common()[:80]]


a = Counter(df_AI_vitals['IF_QO'])
# print(a.most_common(20))
vitals_selected = [x[0] for x in a.most_common()[:12]]

a = Counter(df_AI_medications['M_MedDict'])
# print(a.most_common(30))
meds_selected = [x[0] for x in a.most_common()[:12]]

In [ ]:
### lab preprocess
df_AI_labs['R_ResultDTTM'] = pd.to_datetime(df_AI_labs['R_ResultDTTM'])
df_AI_labs = df_AI_labs[df_AI_labs['IR_QO'].isin(labs_selected)]
lab_names_mapping = {name: i for i, name in enumerate(labs_selected)}
### vital preprocess
PatientID_TO_MRN = {PatientID: MRN for PatientID, MRN in zip(df_AI_labs['PatientID'].to_list(), 
                                                             df_AI_labs['MRN'].to_list())}
df_AI_vitals = df_AI_vitals.replace({"PatientID": PatientID_TO_MRN}).rename(columns={'PatientID': 'MRN'})
df_AI_vitals['F_ClinicalDTTM'] = pd.to_datetime(df_AI_vitals['F_ClinicalDTTM'])
df_AI_vitals = df_AI_vitals[df_AI_vitals['IF_QO'].isin(vitals_selected)]
vital_names_mapping = {name: i for i, name in enumerate(vitals_selected)}
### med preprocess
PatientID_TO_MRN = {PatientID: MRN for PatientID, MRN in zip(df_AI_labs['PatientID'].to_list(), 
                                                             df_AI_labs['MRN'].to_list())}
df_AI_medications = df_AI_medications.replace({"PatientID": PatientID_TO_MRN}).rename(columns={'PatientID': 'MRN'})
df_AI_medications['IM_PerformedDTTM'] = pd.to_datetime(df_AI_medications['IM_PerformedDTTM'])
df_AI_medications = df_AI_medications[df_AI_medications['M_MedDict'].isin(meds_selected)]
med_names_mapping = {name: i for i, name in enumerate(meds_selected)}
### df_AI_demographics 
df_AI_demographics_2 = df_AI_demographics[['MRN', 'Race','Sex']]
df_AI_demographics_2 = pd.get_dummies(df_AI_demographics_2 , columns=['Race','Sex'])
### df_AI_patient_facts 
df_AI_patient_facts_2  = df_AI_patient_facts[['MRN', 'Race', 'Ethnicity']]
df_AI_patient_facts_2  = pd.get_dummies(df_AI_patient_facts_2, columns=['Race', 'Ethnicity'])

# MRI -> MRN
master_sheet = pd.read_excel('../ai_training_set/Updated Database.xlsx', sheet_name = '(6) MRI')
dict_MRI_MRN = {str(int(a)):str(int(b)) for a,b in  zip(master_sheet['ut_scan_id'].to_list(), 
                                                        master_sheet['allscripts_mrn'].to_list()) if not pd.isna(a) and not pd.isna(b)}

# MRI_Train is the MRI Image train set of MRIs
import pickle
with open('embeddings_T2/MRIs_train.pkl', 'rb') as f:
    train_MRIs = pickle.load(f)
with open('embeddings_T2/MRIs_test.pkl', 'rb') as f:
    test_MRIs = pickle.load(f)
T2_embedding_list_test = np.load('embeddings_T2/xgboost_embedding_list_test.npy')
T2_embedding_list_train = np.load('embeddings_T2/xgboost_embedding_list_train.npy')
T2_label_list_test = np.load('embeddings_T2/xgboost_label_list_test.npy')
T2_label_list_train = np.load('embeddings_T2/xgboost_label_list_train.npy')

T1_pre_embedding_list_test = np.load('embeddings_T1_pre/xgboost_embedding_list_test.npy')
T1_pre_embedding_list_train = np.load('embeddings_T1_pre/xgboost_embedding_list_train.npy')
T1_pre_label_list_test = np.load('embeddings_T1_pre/xgboost_label_list_test.npy')
T1_pre_label_list_train = np.load('embeddings_T1_pre/xgboost_label_list_train.npy')

T1_post_embedding_list_test = np.load('embeddings_T1_post/xgboost_embedding_list_test.npy')
T1_post_embedding_list_train = np.load('embeddings_T1_post/xgboost_embedding_list_train.npy')
T1_post_label_list_test = np.load('embeddings_T1_post/xgboost_label_list_test.npy')
T1_post_label_list_train = np.load('embeddings_T1_post/xgboost_label_list_train.npy')

flair_embedding_list_test = np.load('embeddings_flair/xgboost_embedding_list_test.npy')
flair_embedding_list_train = np.load('embeddings_flair/xgboost_embedding_list_train.npy')
flair_label_list_test = np.load('embeddings_flair/xgboost_label_list_test.npy')
flair_label_list_train = np.load('embeddings_flair/xgboost_label_list_train.npy')

pd_embedding_list_test = np.load('embeddings_pd/xgboost_embedding_list_test.npy')
pd_embedding_list_train = np.load('embeddings_pd/xgboost_embedding_list_train.npy')
pd_label_list_test = np.load('embeddings_pd/xgboost_label_list_test.npy')
pd_label_list_train = np.load('embeddings_pd/xgboost_label_list_train.npy')


assert np.array_equal(T2_label_list_train, T1_pre_label_list_train)
assert np.array_equal(T2_label_list_test , T1_pre_label_list_test )
assert np.array_equal( T2_label_list_train , T1_post_label_list_train) 
assert np.array_equal( T2_label_list_test , T1_post_label_list_test)
assert np.array_equal( T2_label_list_train , flair_label_list_train )
assert np.array_equal(  T2_label_list_test , flair_label_list_test)
assert np.array_equal( T2_label_list_train, pd_label_list_train )
assert np.array_equal(  T2_label_list_test , pd_label_list_test)


# Make MRN_train, which corresponds to MRI_Train
train_MRNs = [dict_MRI_MRN[i] for i in train_MRIs]
test_MRNs = [dict_MRI_MRN[i] for i in test_MRIs]

# make it a dictionary
d_mrn_embedding_train_T1_pre = {a:b for a,b in zip(train_MRNs, T1_pre_embedding_list_train)}
d_mrn_embedding_test_T1_pre = {a:b for a,b in zip(test_MRNs, T1_pre_embedding_list_test)}
d_mrn_embedding_train_T1_post = {a:b for a,b in zip(train_MRNs, T1_post_embedding_list_train)}
d_mrn_embedding_test_T1_post = {a:b for a,b in zip(test_MRNs, T1_post_embedding_list_test)}
d_mrn_embedding_train_T2 = {a:b for a,b in zip(train_MRNs, T2_embedding_list_train)}
d_mrn_embedding_test_T2 = {a:b for a,b in zip(test_MRNs, T2_embedding_list_test)}
d_mrn_embedding_train_flair= {a:b for a,b in zip(train_MRNs, flair_embedding_list_train)}
d_mrn_embedding_test_flair = {a:b for a,b in zip(test_MRNs, flair_embedding_list_test)}
d_mrn_embedding_train_pd = {a:b for a,b in zip(train_MRNs, pd_embedding_list_train)}
d_mrn_embedding_test_pd = {a:b for a,b in zip(test_MRNs, pd_embedding_list_test)}

## note embeddings
with open("../../Github_DocumentClassification-MPAD(my-code-Here)/mpad-master/mpad/result_MPAD_embedding.pickle", 'rb') as f:
    note_embeddings = pickle.load(f)
note_embeddings = {a:b for a,b in note_embeddings}


MS_train = []
MS_test = []

for mrn, time_value in tqdm(result_EDSS.items()):
    label = 0
    for time, value in time_value.items():
        time_reach_4 = time
        if value > 4.0: 
            label = 1
            break
        
    ###  ==================== lab ===========================
    n_col = len(lab_names_mapping)
    lab_i = [[0] * n_col]

    patient_lab = df_AI_labs[(df_AI_labs['MRN'] == mrn) & 
                             ((df_AI_labs['R_ResultDTTM'] < time_reach_4))].sort_values(by='R_ResultDTTM').reset_index(drop=True)

    if patient_lab.shape[0] != 0:
        prev_date = patient_lab.loc[0, 'R_ResultDTTM']

        for j in range(patient_lab.shape[0]):
            col = lab_names_mapping[patient_lab.loc[j, 'IR_QO']]
            val = patient_lab.loc[j, 'IR_DecodedValue']
            cur_date = patient_lab.loc[j, 'R_ResultDTTM']
            try: 
                val = float(val)
            except:
                continue
            if np.isnan(val):
                continue

            if cur_date != prev_date:
                lab_i.append([0] * n_col)
                prev_date = cur_date
            lab_i[-1][col] =val


    ###  ==================== vital===========================
    n_col = len(vital_names_mapping)
    vital_i = [[0] * n_col]

    patient_vital = df_AI_vitals[(df_AI_vitals['MRN'] == mrn) & ((df_AI_vitals['F_ClinicalDTTM'] < time_reach_4))].sort_values(by='F_ClinicalDTTM').reset_index(drop=True)

    if patient_vital.shape[0] != 0:
        prev_date = patient_vital.loc[0, 'F_ClinicalDTTM']

        for j in range(patient_vital.shape[0]):
            col = vital_names_mapping[patient_vital.loc[j, 'IF_QO']]
            val = patient_vital.loc[j, 'F_NormalizedValue']
            cur_date = patient_vital.loc[j, 'F_ClinicalDTTM']
            try: 
                val = float(val)
            except:
                continue
            if np.isnan(val):
                continue

            if cur_date != prev_date:
                vital_i.append([0] * n_col)
                prev_date = cur_date
            vital_i[-1][col] =val

    
    ###  ==================== medications ==========================
    n_col = len(med_names_mapping)
    med_i = [[0] * n_col]

    patient_med = df_AI_medications[(df_AI_medications['MRN'] == mrn) & ((df_AI_medications['IM_PerformedDTTM'] < time_reach_4))].sort_values(by='IM_PerformedDTTM').reset_index(drop=True)

    if patient_med.shape[0] != 0:
        prev_date = patient_med.loc[0, 'IM_PerformedDTTM']

        for j in range(patient_med.shape[0]):
            col = med_names_mapping[patient_med.loc[j, 'M_MedDict']]
            val = 1
            cur_date = patient_med.loc[j, 'IM_PerformedDTTM']
            try: 
                val = float(val)
            except:
                continue
            if np.isnan(val):
                continue

            if cur_date != prev_date:
                med_i.append([0] * n_col)
                prev_date = cur_date
            med_i[-1][col] =val

    
    ###  ==================== demographics ==========================
    demo_i_1 = df_AI_demographics_2[df_AI_demographics['MRN']==mrn].drop(columns='MRN').values.tolist()
    if not demo_i_1:
        demo_i_1 = [0] * len(list(df_AI_demographics_2.columns.drop('MRN')))
    else:
        demo_i_1 = demo_i_1[0]
    demo_i_2 = df_AI_patient_facts_2[df_AI_patient_facts['MRN']==mrn].drop(columns='MRN').values.tolist()
    if not demo_i_2:
        demo_i_2 = [0] * len(list(df_AI_patient_facts_2.columns.drop('MRN')))
    else:
        demo_i_2 = demo_i_2[0]
    demo_i = demo_i_1 +  demo_i_2
    
    
    ### ========================== Note Embeddings ====================================
    note_i = note_embeddings[mrn]

    ### ======================= EDSS score ===========================
    label_i = label


    ### ==================== MRI Embeddings ==========================
    ### ===== if MRN has image data and in train =====
    if str(int(mrn)) in d_mrn_embedding_train:
        mri_embed_T1_pre_i = d_mrn_embedding_train_T1_pre[str(int(mrn))]
        mri_embed_T1_post_i = d_mrn_embedding_train_T1_post[str(int(mrn))]
        mri_embed_T2_i = d_mrn_embedding_train_T2[str(int(mrn))]
        mri_embed_flair_i = d_mrn_embedding_train_flair[str(int(mrn))]
        mri_embed_pd_i = d_mrn_embedding_train_pd[str(int(mrn))]
        MS_train.append((str(int(mrn)), (torch.tensor(lab_i, dtype = torch.float32), 
                                         torch.tensor(vital_i, dtype = torch.float32), 
                                         torch.tensor(med_i, dtype = torch.float32), 
                                         torch.tensor(note_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_pre_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_post_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T2_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_flair_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_pd_i, dtype = torch.float32), 
                                         torch.tensor(demo_i, dtype = torch.float32), 
                                         label_i)))
    ### ====== if MRN has image data and in test =======
    elif str(int(mrn)) in d_mrn_embedding_test:
        mri_embed_T1_pre_i = d_mrn_embedding_test_T1_pre[str(int(mrn))]
        mri_embed_T1_post_i = d_mrn_embedding_test_T1_post[str(int(mrn))]
        mri_embed_T2_i = d_mrn_embedding_test_T2[str(int(mrn))]
        mri_embed_flair_i = d_mrn_embedding_test_flair[str(int(mrn))]
        mri_embed_pd_i = d_mrn_embedding_test_pd[str(int(mrn))]
        MS_test.append((str(int(mrn)), (torch.tensor(lab_i, dtype = torch.float32), 
                                         torch.tensor(vital_i, dtype = torch.float32), 
                                         torch.tensor(med_i, dtype = torch.float32), 
                                         torch.tensor(note_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_pre_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_post_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T2_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_flair_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_pd_i, dtype = torch.float32), 
                                         torch.tensor(demo_i, dtype = torch.float32), 
                                         label_i)))
    ### ========= if MRN don't have image, create empty image embedding and put this patient randomly to train/test
    else:
        mri_embed_T1_pre_i = np.zeros(shape=(258,))
        mri_embed_T1_post_i = np.zeros(shape=(258,))
        mri_embed_T2_i = np.zeros(shape=(258,))
        mri_embed_flair_i = np.zeros(shape=(258,))
        mri_embed_pd_i = np.zeros(shape=(258,))
        if np.random.random() < 0.5:
            MS_train.append((str(int(mrn)), (torch.tensor(lab_i, dtype = torch.float32), 
                                         torch.tensor(vital_i, dtype = torch.float32), 
                                         torch.tensor(med_i, dtype = torch.float32), 
                                         torch.tensor(note_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_pre_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_post_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T2_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_flair_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_pd_i, dtype = torch.float32), 
                                         torch.tensor(demo_i, dtype = torch.float32), 
                                         label_i)))
        else:
            MS_test.append((str(int(mrn)), (torch.tensor(lab_i, dtype = torch.float32), 
                                         torch.tensor(vital_i, dtype = torch.float32), 
                                         torch.tensor(med_i, dtype = torch.float32), 
                                         torch.tensor(note_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_pre_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T1_post_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_T2_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_flair_i, dtype = torch.float32), 
                                         torch.tensor(mri_embed_pd_i, dtype = torch.float32), 
                                         torch.tensor(demo_i, dtype = torch.float32), 
                                         label_i)))
            

    
    

In [ ]:
df_AI_demographics_2[df_AI_demographics['MRN']==mrn].drop(columns='MRN')

#### MRI——》MRN是多对1的关系

In [ ]:
len(set(train_MRIs)),\
len(set(test_MRIs)),\
len(set(train_MRIs).intersection(set(test_MRIs)))

In [ ]:
len(set(train_MRNs)),\
len(set(test_MRNs)),\
len(set(train_MRNs).intersection(set(test_MRNs)))

In [ ]:
p = 0
n = 0
na = 0
for mrn, time_value in tqdm(result_EDSS.items()):
    if str(int(mrn)) in d_mrn_embedding_train:
#          print('+')
         p+= 1
    elif str(int(mrn)) in d_mrn_embedding_test:
#          print('-')
         n+=1
    else:
#         print('no')
        na+=1
print(p, n, na)

In [ ]:
len(d_mrn_embedding_train.keys()), \
len(d_mrn_embedding_test.keys()), \
len(set(d_mrn_embedding_train.keys()).intersection(set(d_mrn_embedding_test.keys())))

In [ ]:
d_mrn_embedding_train.keys()

In [ ]:
d_mrn_embedding_test.keys()

In [ ]:
len(MS_train), len(MS_test)

In [ ]:
with open('MS_train.pt', 'wb') as f:
    pickle.dump(MS_train, f)
with open('MS_test.pt', 'wb') as f:
    pickle.dump(MS_test, f)

In [ ]:
import pickle
with open('MS_train.pt', 'rb') as f:
    MS_train = pickle.load(f)
with open('MS_test.pt', 'rb') as f:
    MS_test = pickle.load(f)

In [ ]:
for i in range(10):
    print(MS_train[0][1][i].shape)

In [ ]:
print(len(result_EDSS))
print(len(d_mrn_embedding_train), len(d_mrn_embedding_test))

In [ ]:
d_mrn_embedding_train.keys(), d_mrn_embedding_test.keys()

In [ ]:
print(len(train_MRIs)), print(len(test_MRIs))
print(len(train_MRNs)), print(len(test_MRNs))


In [ ]:
labs_selected[39]

In [ ]:
tmp = []
for i in labs_selected:
#     print("'"+i.split(' ')[0]+"',")
    tmp.append(i)
print(tmp)

tmp = []
for i in vitals_selected:
#     print("'"+i.split(' ')[0]+"',")
    tmp.append(i)
print(tmp)

tmp = []
for i in meds_selected:
#     print("'"+i.split(' ')[0]+"',")
    tmp.append(i)
print(tmp)
